In [2]:
#  Libraries
import numpy as np 
import pandas as pd 
# Data processing, metrics and modeling
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold,KFold
#from bayes_opt import BayesianOptimization
from datetime import datetime
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, roc_curve, auc,precision_recall_curve
from sklearn import metrics
from sklearn import preprocessing
# Lgbm
#import lightgbm as lgb
# Suppr warning
import warnings
warnings.filterwarnings("ignore")

import itertools
from scipy import interp

# Plots
#import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_rows', 100)

## DATASETS

In [4]:
train_transaction = pd.read_csv('train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('test_transaction.csv', index_col='TransactionID')
train_identity = pd.read_csv('train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('test_identity.csv', index_col='TransactionID')
sample_submission = pd.read_csv('sample_submission.csv', index_col='TransactionID')

In [5]:
import gc
gc.collect()

28

## MERGE, MISSING VALUE, FILL NA

In [6]:
# merge 
train_df = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test_df = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

print("Train shape : "+str(train_df.shape))
print("Test shape  : "+str(test_df.shape))

Train shape : (590540, 433)
Test shape  : (506691, 432)


In [63]:
print('starting')
import sys
#sys.path.insert(0 ,"/home/notebooks/goMMT")
from goofy.base.base_utils import logutils as log
from goofy.fileservice import FileServiceHelper
import pandas as pd
import numpy as np
import itertools
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
import tensorflow as tf

from tensorflow import keras
from keras import backend as K, optimizers, Model
from keras.layers import Concatenate, Activation, Dense, Flatten, LSTM, RepeatVector, Dropout, TimeDistributed, GRU, Input
from keras.layers.embeddings import Embedding
from keras.models import model_from_json, Sequential
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ReduceLROnPlateau,CSVLogger
from keras.regularizers import l2
import tensorflow as tf


from dask import dataframe as dd
from dask.multiprocessing import get
from multiprocessing import cpu_count
nCores=cpu_count()

#from QuadricalViews import QuadricalViews
#from MakemytripUtils import *
from QuadricalAdmin import *
from PipelineAdmin import *
from SessionAdmin import *
from FeedFile import *
from QuadricaldbStore import *
#session1.log('this build has session logging working...')
maxfile_count=1000
minfile_rows=100
min_price_delta=10
isNew, useCaseId, pipelineId, file_name,sessionId = (0,1,10016, 'shopclues_combined_full.csv', # '../hugh3/shopclues_financial_clean_large.csv',
                                                     'c05568da-dea5-4deb-a86e-9836b72a711f') ### Shopclues

if isNew==1: sessionId=None
deltaMode=0
db1=QuadricaldbStore()
config1=QuadricalAdmin()
pipeline1=PipelineAdmin(useCaseId,pipelineId)
session1=pipeline1.locateSession(sessionId)
# session1.setcolumnRules(json.loads(config1.getcolumnRules(pipelineId)))
# session1.feedRules = json.loads(config1.getfeedRules(pipelineId))
# sessionId=session1.sessionId
# session1.deltaMode=0
# session1.cachedItems={}
# lag_steps=4
# fut_steps=6
# deltaMode=1
# feedfile1=FeedFile(session1)
# scoringMode=0
# #feedfile1.all_data_df=feedfile1.session1.get_object("gommt-20007-Train-combined-df","pre-target-compute-df-for-varun"+runId,deltaMode=1)
# #feedfile1.process_feed_float_to_category(filterCols=['departure_time','arrival_time'])
# #feedfile1.process_feed_categories_to_hot(deltaMode=0, pipelineId=20007,filterCols=['departure_time', 'arrival_time', 'departure_date_day_of_week','booking_date_day_of_week', 'flight_no_1'])
# #feedfile1.session1.add_object_as_file("gommt-20007-Train-combined-df","pre-target-compute-df-for-varun"+runId,feedfile1.all_data_df)
# #savedf=feedfile1.all_data_df.copy()

starting


Using TensorFlow backend.


Establishing connection with: quadrical-database.mysql.database.azure.com
Connection established
Establishing connection with: quadrical-database.mysql.database.azure.com
Connection established


In [64]:
train_df_added = session1.get_object('ae_reconstruction_error_811', 'train_df_added') # not work
test_df_added = session1.get_object('ae_reconstruction_error_811', 'test_df_added')

train_df_added.index = train_df.index
test_df_added.index = test_df.index

train_df = pd.concat([train_df, train_df_added], axis = 1)
test_df = pd.concat([test_df, test_df_added], axis = 1)

reading file 58ea243e-6bbe-40a2-9b72-aaf902fc16f1
reading file 6be49d98-9969-4ede-975c-9caea5f93490


In [7]:
# New groupby
train_df['days_passed'] = train_df['TransactionDT'] // 86400
test_df['days_passed'] = test_df['TransactionDT'] // 86400
train_df['days_created'] = train_df['days_passed'] - train_df['D1']
test_df['days_created'] = test_df['days_passed'] - test_df['D1']

train_df['uid4'] = train_df['card1'].astype(str) + '_' + train_df['ProductCD'].astype(str) +'_' + train_df['P_emaildomain'].astype(str) + '_' + \
                train_df['addr1'].astype(str) + '_' + train_df['days_created'].astype(str)
test_df['uid4'] = test_df['card1'].astype(str) + '_' + test_df['ProductCD'].astype(str) +'_' + test_df['P_emaildomain'].astype(str) + '_' + \
                test_df['addr1'].astype(str) + '_' + test_df['days_created'].astype(str)

In [ ]:
def add_GroupBy_mean(df, by,  window = 10, target = 'TransactionAmt'):
    for fea_by in by:
        print('Processing', fea_by)
        #df[fea_by + target +'minus_mean_window'] =  df[target] - df.groupby(fea_by)[target].transform(lambda x: x.rolling(window, 1).mean())
        df[fea_by + target +'minus_mean_all'] =   df[target] - df.groupby(fea_by)[target].transform('mean')
    
    return df

In [ ]:
num_list = ['TransactionAmt', 'D15']

In [12]:
num_list  =['TransactionAmt',
 'dist1',
 'dist2',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'D10',
 'D11',
 'D12',
 'D13',
 'D14',
 'D15',
 'id_02',
 'id_05',
 'id_06',
 'nulls1',
 'V307',
 'V310',
 'V313',
 'C5_fq_enc',
 'P_emaildomain_bin',
 'card5_count_full',
 'card5_TransactionAmt_std',
 'V315',
 'id_02_to_mean_card1',
 'C11_fq_enc',
 'C9_fq_enc',
 'D15_to_mean_addr1',
 'D15_to_mean_card4',
 'card5_TransactionAmt_mean',
 'C2_fq_enc',
 'D15_to_std_card1',
 'C14_fq_enc',
 'C6_fq_enc',
 'id_31_count_dist',
 'Transaction_day_of_week',
 'C1_fq_enc',
 'uid_TransactionAmt_std',
 'D15_to_mean_card1',
 'TransactionAmt_decimal',
 'card2_TransactionAmt_std',
 'TransactionAmt_to_mean_card4',
 'uid_TransactionAmt_mean',
 'C13_fq_enc',
 'Transaction_hour_of_day',
 'TransactionAmt_to_std_card1',
 'TransactionAmt_to_std_card4',
 'card1_TransactionAmt_std',
 'card2_TransactionAmt_mean',
 'V13TransactionAmtminus_mean_all',
 'TransactionAmt_to_mean_card1',
 'card1_TransactionAmt_mean',
 'uid2_TransactionAmt_std',
 'card2_fq_enc',
 'uid2_TransactionAmt_mean',
 'card1_count_full']

In [9]:
train_df_added = pd.read_pickle('train_df.pkl')

In [18]:
test_df_added = pd.read_pickle('test_df.pkl')

In [10]:
train_df_added.shape

(590540, 791)

In [16]:
features_added = []

for fea in num_list:
    if fea in train_df_added and fea not in train_df:
        #print(fea)
        features_added.append(fea)

In [19]:
train_df_added = train_df_added[features_added]
test_df_added = test_df_added[features_added]

In [20]:
features_added = [fea for fea in train_df_added if fea not in train_df]

train_df_added = train_df_added[features_added]
test_df_added = test_df_added[features_added]

train_df_added.index = train_df.index
test_df_added.index = test_df.index

train_df = pd.concat([train_df, train_df_added], axis = 1)
test_df = pd.concat([test_df, test_df_added], axis = 1)

In [21]:
train_df.shape
test_df.shape

(590540, 455)

(506691, 454)

In [24]:
num_list = [fea for fea in num_list if fea in train_df ]

In [25]:
num_list

['TransactionAmt',
 'dist1',
 'dist2',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'D10',
 'D11',
 'D12',
 'D13',
 'D14',
 'D15',
 'id_02',
 'id_05',
 'id_06',
 'V307',
 'V310',
 'V313',
 'C5_fq_enc',
 'card5_TransactionAmt_std',
 'V315',
 'C11_fq_enc',
 'C9_fq_enc',
 'card5_TransactionAmt_mean',
 'C2_fq_enc',
 'C14_fq_enc',
 'C6_fq_enc',
 'C1_fq_enc',
 'uid_TransactionAmt_std',
 'card2_TransactionAmt_std',
 'uid_TransactionAmt_mean',
 'C13_fq_enc',
 'card1_TransactionAmt_std',
 'card2_TransactionAmt_mean',
 'card1_TransactionAmt_mean',
 'uid2_TransactionAmt_std',
 'card2_fq_enc',
 'uid2_TransactionAmt_mean']

In [26]:
del train_df['isFraud']

In [71]:
train_df.shape
test_df.shape

(590540, 455)

(506691, 455)

In [72]:
data = pd.concat([train_df, test_df])

In [73]:
data.shape

(1097231, 455)

In [74]:
data.columns

Index(['TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2',
       'card3', 'card4', 'card5', 'card6', 'addr1',
       ...
       'card2_TransactionAmt_std', 'uid_TransactionAmt_mean', 'C13_fq_enc',
       'card1_TransactionAmt_std', 'card2_TransactionAmt_mean',
       'card1_TransactionAmt_mean', 'uid2_TransactionAmt_std', 'card2_fq_enc',
       'uid2_TransactionAmt_mean', 'ae_reconstruction_error'],
      dtype='object', length=455)

In [75]:
import gc
gc.collect()

43

In [76]:
# fea_by_list = ['uid4']
# target_list = num_list
target_list = ['ae_reconstruction_error']

In [77]:
len(fea_by_list)
len(target_list)

1

1

In [78]:
columns_name = []
added = []

for target in target_list:
    for fea_by in fea_by_list:
        print('Now Processing', target, fea_by)
        added.append(  data[target] - data.groupby(fea_by)[target].transform('mean')  )
        added.append(  data[target] / data.groupby(fea_by)[target].transform('mean')  )
        added.append(  data[target] / data.groupby(fea_by)[target].transform('std')  )
        columns_name.append(fea_by + '_' + target +'_' + '_minus_mean')
        columns_name.append(fea_by + '_' + target +'_' + '_divide_mean')
        columns_name.append(fea_by + '_' + target +'_' + '_divide_std')

Now Processing ae_reconstruction_error uid4


In [79]:
data_added = pd.DataFrame( np.stack(added, axis = 1) )
data_added.columns = columns_name

data_added.shape
data_added.head()

(1097231, 3)

,uid4_ae_reconstruction_error__minus_mean,uid4_ae_reconstruction_error__divide_mean,uid4_ae_reconstruction_error__divide_std
0,0.000000,1.000000,NaN
1,0.000000,1.000000,NaN
2,-0.000231,0.930559,12.270024
3,0.001134,1.108609,2.952558
4,0.000000,1.000000,NaN


In [56]:
from copy import deepcopy

In [57]:
data_added_back = deepcopy(data_added)

In [58]:
data_added_back.shape

(1097231, 174)

In [59]:
data_added.to_pickle('added_Groupby_divide&minus_uid4')

In [82]:
len(fea_by_list)
len(target_list)

1

1

In [83]:
columns_name = []
added = []

for target in target_list:
    for fea_by in fea_by_list:
        print('Now Processing', target, fea_by)
        added.append(data.groupby(fea_by)[target].transform('mean'))
        added.append(data.groupby(fea_by)[target].transform('std'))
        columns_name.append(fea_by + '_' + target +'_' + 'mean')
        columns_name.append(fea_by + '_' + target +'_' + 'std')

Now Processing ae_reconstruction_error uid4


In [84]:
data_added = pd.DataFrame( np.stack(added, axis = 1) )
data_added.columns = columns_name

data_added.shape
data_added.head()

(1097231, 2)

,uid4_ae_reconstruction_error_mean,uid4_ae_reconstruction_error_std
0,0.006511,NaN
1,0.001674,NaN
2,0.003331,0.000253
3,0.010437,0.003919
4,0.009007,NaN


In [66]:
data_added.to_pickle('Groupby_uid4')

In [85]:
data_added_back = pd.concat([data_added, data_added_back], axis = 1)

In [88]:
data_added_back.shape

(1097231, 295)

In [89]:
data_added_back.to_pickle('all_uid4')

In [ ]:
oof_lgb = session1.get_object('25pred', 'oof')
predictions_lgb = session1.get_object('25pred', 'predictions')

train_df['pred'] = oof_lgb
test_df['pred'] = predictions_lgb

In [ ]:
# invest = ['ProductCD', 'card1', 'card2','card3','card4','card5','card6','dist1', 'dist2',
#           'addr1','addr2',
#           'P_emaildomain',
#           'R_emaildomain']

# for fea in invest:
#     print(fea)
#     train_df[fea] = train_df[fea].astype(str)
#     test_df[fea] = test_df[fea].astype(str)

In [ ]:
invest = ['ProductCD', 'card1', 'card2','card3','card4','card5','card6','dist1', 'dist2',
          'addr1','addr2',
          'P_emaildomain',
          'R_emaildomain']

for fea in invest:
    print(fea, train_df[fea].isnull().sum())

In [ ]:
import datetime

In [ ]:
train_df['days_passed'] = train_df['TransactionDT'] // 86400
train_df['days_created'] = train_df['days_passed']  - train_df['D1']

In [ ]:
train_df['uid4'] = train_df['card1'].astype(str)+'_'+train_df['ProductCD'].astype(str)+ '_'+train_df['P_emaildomain'].astype(str)+'_'+train_df['addr1'].astype(str)+'_' + train_df['days_created'].astype(str) # 9479

In [ ]:
train_df['uid4'].nunique()

In [ ]:
train_df['uid4'][train_df.isFraud == 1].value_counts()

In [ ]:
pd.set_option('display.max_rows', 1000)

In [ ]:
#train_df['isFraud'][ train_df['uid4'] == '12577_W_gmail.com_299.0_150.0']

In [ ]:
train_df['uid4'].isnull().sum() # only this 0.9479
train_df.loc[train_df['addr1'].isnull(), 'uid4'] = np.nan
train_df['uid4'].isnull().sum()

In [ ]:
# train_df['uid4'].isnull().sum() # do not add this 9471
# train_df.loc[train_df['P_emaildomain'].isnull(), 'uid4'] = np.nan
# train_df['uid4'].isnull().sum()

In [ ]:
transform(lambda x: x.rolling(window, 1).count())

In [ ]:
train_df['ProductCD']

In [ ]:
train_df['uid4'].isnull().sum()

In [ ]:
window = 5
temp_dict_prev = train_df.groupby('uid4')['pred'].rolling(window, 1).mean().to_dict()

In [ ]:
window = 5
temp_dict_back = train_df[::-1].groupby('uid4')['pred'].rolling(window, 1).mean().to_dict()

In [ ]:
train_df['index'] = train_df.index

In [ ]:
train_df['index']

In [ ]:
train_df['sep'] = '&'

In [ ]:
train_df['uid4+index'] = train_df['uid4'] + train_df['sep']  +  train_df['index'].astype(str)

In [ ]:
def tmp(x):
    return (x.split('&')[0], int(x.split('&')[1]))

In [ ]:
train_df['uid4+index'][~train_df['uid4+index'].isnull()] = train_df['uid4+index'][~train_df['uid4+index'].isnull()].apply(tmp)

In [ ]:
temp_dict_prev

In [ ]:
# 9482
temp_dict = train_df.groupby('uid4')['pred'].mean().to_dict()
train_df.loc[~(train_df['uid4'].isnull()), 'pred2'] = 0.15 * train_df['uid4+index'].map(temp_dict_back) + 0.15 * train_df['uid4+index'].map(temp_dict_prev) + 0.7 * train_df['uid4'].map(temp_dict)
#train_df.loc[~(train_df['uid4'].isnull()), 'pred2'] = train_df['uid4+index'].map(temp_dict_prev)
train_df.loc[train_df['uid4'].isnull(), 'pred2'] = train_df['pred']

In [ ]:
# 9482
#temp_dict = train_df.groupby('uid4')['pred'].mean().to_dict()
train_df.loc[~(train_df['uid4'].isnull()), 'pred2'] = 0.5 * train_df['uid4+index'].map(temp_dict_back) + 0.5 * train_df['uid4+index'].map(temp_dict_prev)
#train_df.loc[~(train_df['uid4'].isnull()), 'pred2'] = train_df['uid4+index'].map(temp_dict_prev)
train_df.loc[train_df['uid4'].isnull(), 'pred2'] = train_df['pred']

In [ ]:
train_df['uid4'].isnull().sum()

In [ ]:
train_df['pred2'].isnull().sum()

In [ ]:
temp_dict = train_df.groupby('uid4')['pred'].mean().to_dict()
train_df.loc[~(train_df['uid4'].isnull()), 'pred2'] = train_df['uid4'].map(temp_dict)
train_df.loc[train_df['uid4'].isnull(), 'pred2'] = train_df['pred']

In [ ]:
gc.collect()

In [ ]:
    features = ['pred2']

    start = time()
    plt.rcParams["axes.grid"] = True
    nfold = 5
    #skf = StratifiedKFold(n_splits=nfold, shuffle=True, random_state=42)
    skf = KFold(n_splits=nfold, shuffle=False, random_state=42)

    oof = np.zeros(len(train_df))
    mean_fpr = np.linspace(0,1,100)
    cms= []
    tprs = []
    aucs = []
    y_real = []
    y_proba = []
    recalls = []
    roc_aucs = []
    f1_scores = []
    accuracies = []
    precisions = []
    predictions = np.zeros(len(test_df))
    feature_importance_df = pd.DataFrame()

    i = 1
#     clf = lgb.LGBMClassifier()
#     clf.set_params(**param_lgb)
    #clf = LogisticRegression()
    
    
    for train_idx, valid_idx in skf.split(train_df, train_df.isFraud.values):
        print("\nfold {}".format(i))
        x_train = train_df.iloc[train_idx][features]
        y_train = train_df.iloc[train_idx][target]

        x_val = train_df.iloc[valid_idx][features]
        y_val = train_df.iloc[valid_idx][target]


#         clf = lgb.LGBMClassifier()
#         clf.set_params(**param_lgb)
#         clf.fit(x_train, y_train,
#                 eval_metric =['auc'],
#                 verbose = 10,
#                 eval_set = (x_val, y_val)) 


        oof[valid_idx] = x_val['pred2']

        #predictions += clf.predict_proba(test_df[features])[:,1]  / nfold


        # Scores 
        print(roc_auc_score(train_df.iloc[valid_idx][target].values, oof[valid_idx]))
        
        roc_aucs.append(roc_auc_score(train_df.iloc[valid_idx][target].values, oof[valid_idx]))
        accuracies.append(accuracy_score(train_df.iloc[valid_idx][target].values, oof[valid_idx].round()))
        recalls.append(recall_score(train_df.iloc[valid_idx][target].values, oof[valid_idx].round()))
        precisions.append(precision_score(train_df.iloc[valid_idx][target].values ,oof[valid_idx].round()))
        f1_scores.append(f1_score(train_df.iloc[valid_idx][target].values, oof[valid_idx].round()))

        # Roc curve by folds
        f = plt.figure(1)
        fpr, tpr, t = roc_curve(train_df.iloc[valid_idx][target].values, oof[valid_idx])
        tprs.append(interp(mean_fpr, fpr, tpr))
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.4f)' % (i,roc_auc))

        # Precion recall by folds
        g = plt.figure(2)
        precision, recall, _ = precision_recall_curve(train_df.iloc[valid_idx][target].values, oof[valid_idx])
        y_real.append(train_df.iloc[valid_idx][target].values)
        y_proba.append(oof[valid_idx])
        plt.plot(recall, precision, lw=2, alpha=0.3, label='P|R fold %d' % (i))  

        i= i+1

        # Confusion matrix by folds
        cms.append(confusion_matrix(train_df.iloc[valid_idx][target].values, oof[valid_idx].round()))

#         # Features imp
#         fold_importance_df = pd.DataFrame()
#         fold_importance_df["Feature"] = features
#         fold_importance_df["importance"] = pd.DataFrame(sorted(zip(clf.feature_importances_,x_train.columns)), 
#                                                         columns=['Value','Feature'])['Value']
#         fold_importance_df["fold"] = nfold + 1
#         feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        

        # Metrics
    print(
        '\nCV roc score        : {0:.4f}, std: {1:.4f}.'.format(np.mean(roc_aucs), np.std(roc_aucs)),
        '\nCV accuracy score   : {0:.4f}, std: {1:.4f}.'.format(np.mean(accuracies), np.std(accuracies)),
        '\nCV recall score     : {0:.4f}, std: {1:.4f}.'.format(np.mean(recalls), np.std(recalls)),
        '\nCV precision score  : {0:.4f}, std: {1:.4f}.'.format(np.mean(precisions), np.std(precisions)),
        '\nCV f1 score         : {0:.4f}, std: {1:.4f}.'.format(np.mean(f1_scores), np.std(f1_scores))
        )
    length = time() - start

In [ ]:
START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')

train_df['TransactionDT'] = train_df['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))

train_df['Date'] = train_df['TransactionDT'].apply(lambda x: str(x)[:10])

In [ ]:
test_df['TransactionDT'] = test_df['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))

test_df['Date'] = test_df['TransactionDT'].apply(lambda x: str(x)[:10])

In [ ]:
train_df['uid4'] = train_df['card1'].astype(str)+'_'+train_df['ProductCD'].astype(str)+ '_'+train_df['P_emaildomain'].astype(str)+'_'+train_df['addr1'].astype(str)+'_'

In [ ]:
test_df['uid4'] = test_df['card1'].astype(str)+'_'+test_df['ProductCD'].astype(str)+ '_'+test_df['P_emaildomain'].astype(str)+'_'+test_df['addr1'].astype(str)+'_'

In [ ]:
test_df['uid4'].isnull().sum()

In [ ]:
train_df.loc[train_df['addr1'].isnull(), 'uid4'] = np.nan
test_df.loc[test_df['addr1'].isnull(), 'uid4'] = np.nan

In [ ]:
train_df.loc[train_df['P_emaildomain'].isnull(), 'uid4'] = np.nan
test_df.loc[test_df['P_emaildomain'].isnull(), 'uid4'] = np.nan

In [ ]:
test_df['uid4'].isnull().sum()

In [ ]:
temp_dict = train_df.groupby('uid4')['pred'].mean().to_dict()

In [ ]:
temp_dict

In [ ]:
train_df['pred2'] = 0

In [ ]:
train_df.loc[~(train_df['uid4'].isnull()), 'pred2'] = train_df['uid4'].map(temp_dict)
train_df.loc[train_df['uid4'].isnull(), 'pred2'] = train_df['pred']

In [ ]:
target = 'isFraud'

In [ ]:
features = ['pred']

In [ ]:
train_df.shape
test_df.shape

In [ ]:
oof_lgb = session1.get_object('25pred', 'oof')
predictions_lgb = session1.get_object('25pred', 'predictions')

In [ ]:
train_df['pred'] = oof_lgb
test_df['pred'] = predictions_lgb

In [ ]:
train_df.shape
test_df.shape

In [ ]:
from time import time

In [ ]:
features = ['pred']

In [ ]:
    start = time()
    plt.rcParams["axes.grid"] = True
    nfold = 5
    #skf = StratifiedKFold(n_splits=nfold, shuffle=True, random_state=42)
    skf = KFold(n_splits=nfold, shuffle=False, random_state=42)

    oof = np.zeros(len(train_df))
    mean_fpr = np.linspace(0,1,100)
    cms= []
    tprs = []
    aucs = []
    y_real = []
    y_proba = []
    recalls = []
    roc_aucs = []
    f1_scores = []
    accuracies = []
    precisions = []
    predictions = np.zeros(len(test_df))
    feature_importance_df = pd.DataFrame()

    i = 1
#     clf = lgb.LGBMClassifier()
#     clf.set_params(**param_lgb)
    #clf = LogisticRegression()
    
    
    for train_idx, valid_idx in skf.split(train_df, train_df.isFraud.values):
        print("\nfold {}".format(i))
        x_train = train_df.iloc[train_idx][features]
        y_train = train_df.iloc[train_idx][target]

        x_val = train_df.iloc[valid_idx][features]
        y_val = train_df.iloc[valid_idx][target]


#         clf = lgb.LGBMClassifier()
#         clf.set_params(**param_lgb)
#         clf.fit(x_train, y_train,
#                 eval_metric =['auc'],
#                 verbose = 10,
#                 eval_set = (x_val, y_val)) 


        oof[valid_idx] = x_val['pred']

        #predictions += clf.predict_proba(test_df[features])[:,1]  / nfold


        # Scores 
        print(roc_auc_score(train_df.iloc[valid_idx][target].values, oof[valid_idx]))
        
        roc_aucs.append(roc_auc_score(train_df.iloc[valid_idx][target].values, oof[valid_idx]))
        accuracies.append(accuracy_score(train_df.iloc[valid_idx][target].values, oof[valid_idx].round()))
        recalls.append(recall_score(train_df.iloc[valid_idx][target].values, oof[valid_idx].round()))
        precisions.append(precision_score(train_df.iloc[valid_idx][target].values ,oof[valid_idx].round()))
        f1_scores.append(f1_score(train_df.iloc[valid_idx][target].values, oof[valid_idx].round()))

        # Roc curve by folds
        f = plt.figure(1)
        fpr, tpr, t = roc_curve(train_df.iloc[valid_idx][target].values, oof[valid_idx])
        tprs.append(interp(mean_fpr, fpr, tpr))
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.4f)' % (i,roc_auc))

        # Precion recall by folds
        g = plt.figure(2)
        precision, recall, _ = precision_recall_curve(train_df.iloc[valid_idx][target].values, oof[valid_idx])
        y_real.append(train_df.iloc[valid_idx][target].values)
        y_proba.append(oof[valid_idx])
        plt.plot(recall, precision, lw=2, alpha=0.3, label='P|R fold %d' % (i))  

        i= i+1

        # Confusion matrix by folds
        cms.append(confusion_matrix(train_df.iloc[valid_idx][target].values, oof[valid_idx].round()))

#         # Features imp
#         fold_importance_df = pd.DataFrame()
#         fold_importance_df["Feature"] = features
#         fold_importance_df["importance"] = pd.DataFrame(sorted(zip(clf.feature_importances_,x_train.columns)), 
#                                                         columns=['Value','Feature'])['Value']
#         fold_importance_df["fold"] = nfold + 1
#         feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        

        # Metrics
    print(
        '\nCV roc score        : {0:.4f}, std: {1:.4f}.'.format(np.mean(roc_aucs), np.std(roc_aucs)),
        '\nCV accuracy score   : {0:.4f}, std: {1:.4f}.'.format(np.mean(accuracies), np.std(accuracies)),
        '\nCV recall score     : {0:.4f}, std: {1:.4f}.'.format(np.mean(recalls), np.std(recalls)),
        '\nCV precision score  : {0:.4f}, std: {1:.4f}.'.format(np.mean(precisions), np.std(precisions)),
        '\nCV f1 score         : {0:.4f}, std: {1:.4f}.'.format(np.mean(f1_scores), np.std(f1_scores))
        )
    length = time() - start

In [ ]:
train_df.head(100)

In [ ]:
train_df['TransactionAmt']

In [ ]:
invest = ['ProductCD', 'card1', 'card2','card3','card4','card5','card6','dist1', 'dist2',
          'addr1','addr2',
          'P_emaildomain',
          'R_emaildomain']

for fea in invest:
    print(fea)
    train_df[fea] = train_df[fea].astype(str)
    test_df[fea] = test_df[fea].astype(str)
    

train_df['uid4'] = train_df['uid3'].astype(str)+'_'+train_df['P_emaildomain'].astype(str)
test_df['uid4'] = test_df['uid3'].astype(str)+'_'+test_df['P_emaildomain'].astype(str)

In [ ]:
train_df['uid'] = train_df['card1'].astype(str)+'_'+train_df['card2'].astype(str)
test_df['uid'] = test_df['card1'].astype(str)+'_'+test_df['card2'].astype(str)

train_df['uid2'] = train_df['uid'].astype(str)+'_'+train_df['card3'].astype(str)+'_'+train_df['card5'].astype(str)
test_df['uid2'] = test_df['uid'].astype(str)+'_'+test_df['card3'].astype(str)+'_'+test_df['card5'].astype(str)

train_df['uid3'] = train_df['uid2'].astype(str)+'_'+train_df['addr1'].astype(str)+'_'+train_df['addr2'].astype(str)
test_df['uid3'] = test_df['uid2'].astype(str)+'_'+test_df['addr1'].astype(str)+'_'+test_df['addr2'].astype(str)

train_df['uid4'] = train_df['uid3'].astype(str)+'_'+train_df['P_emaildomain'].astype(str)
test_df['uid4'] = test_df['uid3'].astype(str)+'_'+test_df['P_emaildomain'].astype(str)

train_df['uid5'] = train_df['uid3'].astype(str)+'_'+train_df['R_emaildomain'].astype(str)
test_df['uid5'] = test_df['uid3'].astype(str)+'_'+test_df['R_emaildomain'].astype(str)

In [ ]:
train_df['my_uid1'] = train_df['card1'] + train_df['card2']+ train_df['card3']+ train_df['card4']+ train_df['card5']+ train_df['card6']
train_df['my_uid2'] = train_df['my_uid1'] + train_df['addr1']+ train_df['addr2'] 
train_df['my_uid3'] = train_df['my_uid2'] + train_df['P_emaildomain'] + train_df['R_emaildomain']
train_df['my_uid4'] = train_df['my_uid3'] + train_df['dist1'] + train_df['dist2']

In [ ]:
test_df['my_uid1'] = test_df['card1'] + test_df['card2']+ test_df['card3']+ test_df['card4']+ test_df['card5']+ test_df['card6']
test_df['my_uid2'] = test_df['my_uid1'] + test_df['addr1']+ test_df['addr2'] 
test_df['my_uid3'] = test_df['my_uid2'] + test_df['P_emaildomain'] + test_df['R_emaildomain']
test_df['my_uid4'] = test_df['my_uid3'] + test_df['dist1'] + test_df['dist2']

In [ ]:
pre_list = ['ProductCD', 'card1', 'card2','card3','card4','card5','card6','dist1', 'dist2',
          'addr1','addr2',
          'P_emaildomain',
          'R_emaildomain']

In [ ]:
pre_list += list(train_df.columns[-9:])

In [ ]:
pre_list

In [ ]:
train_df.head()

In [ ]:
train_df[['TransactionAmt','TransactionDT','D15']].isnull().sum()

In [ ]:
train_df['D15'].fillna(999999, inplace =True)
test_df['D15'].fillna(9999, inplace =True)

In [ ]:
for target in ['TransactionAmt','TransactionDT','D15']:
    for fea in pre_list:
        print(fea, target)
        train_df[fea + target + 'rank'] = train_df.groupby([fea])[target].rank(method='dense').astype(int)
        test_df[fea + target + 'rank'] = test_df.groupby([fea])[target].rank(method='dense').astype(int)

In [ ]:
use_list = list(train_df.columns[-len(pre_list)*3:])

use_list

In [ ]:
len(use_list)

train_df_added = train_df[use_list]
test_df_added = test_df[use_list]

train_df_added.head()
test_df_added.head()

train_df_added.shape
test_df_added.shape

In [ ]:
train_df_added = train_df_added.reset_index().iloc[:,1:]
test_df_added = test_df_added.reset_index().iloc[:,1:]

gc.collect()

train_df_added.head()
test_df_added.head()

In [ ]:
session1.add_object_as_file('rank', 'train_df_added', train_df_added)
session1.add_object_as_file('rank', 'test_df_added', test_df_added)